In [1]:
import requests
import time
import json

In [356]:
def get_klinedata(platform, symbol, granularity):
    klinedata = []
    try:
        # [时间，开盘价，最高价，最低价，收盘价，交易量]
        if platform == "okex":  # 200条数据，时间粒度60,300,3600,86400……
            res = requests.get("https://www.okex.com/api/spot/v3/instruments/{}/candles?granularity={}".format(
                symbol.upper().replace("_", "-"), granularity), timeout=1)
            klinedata = json.loads(res.content.decode())[::-1]  # 时间升序排列
            for i in klinedata:
                t = i[0].replace("T", " ").replace(".000Z", "")
                timeStruct = time.strptime(t, "%Y-%m-%d %H:%M:%S")
                timeStamp = int(time.mktime(timeStruct)) + 60 * 60 * 8
                i[0] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timeStamp))
        if platform == "huobi":  # 200条数据，时间粒度1min, 5min, 15min, 30min, 60min, 4hour, 1day, 1mon, 1week, 1year
            huobi_granularity_dict = {60: "1min", 300: "5min", 900: "15min", 1800: "30min", 3600: "60min",
                                      14400: "4hour", 86400: "1day", 604800: "1week", 2592000: "mon",
                                      946080000: "1year"}
            res = requests.get("https://api.huobi.pro/market/history/kline?period={}&size=1000&symbol={}".format(
                huobi_granularity_dict[granularity], symbol.replace("_", "")), timeout=1)
            data = json.loads(res.content.decode())["data"]
            klinedata = []
            for i in data:
#                 l = [
#                     time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(i["id"])),
#                     i["open"],
#                     i["high"],
#                     i["low"],
#                     i["close"],
#                     i["amount"]
#                 ]
                l=i['close']
                klinedata.append(l)
            klinedata = klinedata[::-1]
        if platform == "binance":  # 200条数据，时间粒度1m, 5m, 15m, 30m, 1h, 4h, 1d
            binance_granularity_dict = {60: "1m", 300: "5m", 900: "15m", 1800: "30m", 3600: "1h",
                                        14400: "4h", 86400: "1d"}
            res = requests.get("https://www.binancezh.cc/api/v3/klines?symbol={}&interval={}&limit=200".format(
                symbol.upper().replace("_", ""), binance_granularity_dict[granularity]), timeout=1)
            data = json.loads(res.content.decode())
            klinedata = []
            for i in data:
                l = [
                    time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(str(i[0])[:10]))),
                    i[1],
                    i[2],
                    i[3],
                    i[4],
                    i[5],
                ]
                klinedata.append(l)
    except:
        pass
    finally:
        symbol_klinedata = (symbol, klinedata)
    return symbol_klinedata

In [340]:
eth_usdt=get_klinedata('huobi', 'btc_usdt',86400)  #p1

In [341]:
eos_usdt=get_klinedata('huobi','aion_usdt',86400) #p3

In [342]:
eos_eth=get_klinedata('huobi','aion_btc',86400) #p2

In [343]:
eth_usdt

('btc_usdt',
 [['2018-09-20 00:00:00',
   6329.89,
   6518.67,
   6121.83,
   6415.36,
   15575.420946222057],
  ['2018-09-21 00:00:00',
   6415.36,
   6798.5,
   6338.13,
   6684.58,
   22460.49356460057],
  ['2018-09-22 00:00:00',
   6684.19,
   6839.25,
   6633.9,
   6683.95,
   14640.158716397667],
  ['2018-09-23 00:00:00',
   6685.93,
   6785.0,
   6640.96,
   6688.32,
   8823.546563513337],
  ['2018-09-24 00:00:00', 6688.32, 6729.08, 6577.0, 6611.46, 8809.6412944534],
  ['2018-09-25 00:00:00',
   6611.29,
   6663.14,
   6365.0,
   6404.95,
   12537.01547988195],
  ['2018-09-26 00:00:00',
   6406.0,
   6560.05,
   6331.0,
   6539.64,
   10373.620886787296],
  ['2018-09-27 00:00:00', 6542.03, 6550.0, 6435.0, 6498.04, 9306.149649427898],
  ['2018-09-28 00:00:00',
   6499.25,
   6800.0,
   6496.97,
   6672.3,
   16210.750113268437],
  ['2018-09-29 00:00:00', 6672.31, 6695.0, 6467.85, 6591.4, 10614.03380774986],
  ['2018-09-30 00:00:00',
   6589.89,
   6664.99,
   6548.45,
   6605.18,

In [344]:
eth_usdt=[i[4] for i in eth_usdt[1]]

In [345]:
eos_usdt=[i[4] for i in eos_usdt[1]]

In [346]:
eos_eth=[i[4] for i in eos_eth[1]]

In [350]:
len(eth_usdt)

1000

In [351]:
len(eos_usdt)

0

In [352]:
len(eos_eth)

0

In [339]:
diff1=[]
diff2=[]
for i in range(len(eth_usdt)):
    diff1.append(eos_usdt[i]*(0.998**3)-eth_usdt[i]*eos_eth[i])
    diff2.append(eth_usdt[i]*eos_eth[i]*(0.998**3)-eos_usdt[i])

IndexError: list index out of range

In [327]:
diff1

[0.1969476237560004,
 0.0016000222400007758,
 0.2514232350128003,
 -0.01144627493439998,
 -0.0424484134872003,
 0.13537415631999927,
 -0.011173920080000066,
 -0.01661035540000011,
 0.06107946289519983,
 0.21838310640000103,
 -0.024426822848000462,
 0.05237781023119936,
 -0.027836210355999924,
 0.07045498655999971,
 0.040565822848799726,
 0.00876667398719988,
 0.02375874497039998,
 -0.02017940316880029,
 -0.010229057080799464,
 -0.0038570014967991284,
 -0.02096999559920043,
 0.09898542879999983,
 0.07349373268480086,
 0.035943017457599424,
 -0.025535393771999537,
 0.2526617588000004,
 0.09893029030080047,
 0.05554953317519917,
 0.07782599597600015,
 -0.009655682453599823,
 0.02917344296000035,
 -0.004089862248799925,
 -0.011113928360000358,
 0.029079545907200632,
 0.015138115819199882,
 0.0033687687919998766,
 0.012039786098399397,
 -0.010317883199999933,
 0.006776675511999564,
 0.012530457734399647,
 -0.02005827288239992,
 0.03215159251119992,
 -0.00418014493999852,
 -0.026708423348000

In [80]:
t=[]
for i in range(len(diff1)):
    if diff1[i]>0.05:
        print(eth_usdt[i])
        print(i,eth_usdt[i])
        t.append(i)

In [127]:
threshold=[i for i in diff2 if i>0.01]


In [128]:
len(threshold)

0

In [53]:
eos_eth[-11] 2

0.0023912

In [54]:
eos_usdt[-11] 3

5.7411

In [55]:
eth_usdt[-11] 1

2272.0

In [68]:
5.7411*(0.998**3)-2272*0.0023912

0.27391584727120044

In [56]:
2272/5.7411*0.0023912

0.9463006044137883

In [ ]:
# 2272 usdt--->1eth                             eth_usdt 2272  
# 1    eth --->1/0.0023912 eos                  eos_eth  0.0023912      0.998---
# 1/0.0023912 eos----->2400.928usdt              eos_usdt 5.7411

In [60]:
diff.index(1.0126126126126127)

258

In [61]:
eos_eth[258] 3

0.0281

In [62]:
eos_usdt[258] 2 

6.66

In [63]:
eth_usdt[258] 1

240.0

In [64]:
240.0*0.0281/6.66

1.0126126126126125

In [ ]:
#反向套利交易顺序
# 666 usdt ----》 100 EOS   eos_usdt 6.66  
# 100 EOS  ----》   2.81ETH        eos_eth  0.0281
# 2.81 ETH -----》 674 ust   eth_usdt 240

In [58]:
#(P2-P1*P3)*Q3>Q3*tax*(2*P2+P1*P3)
5.7411-2272*0.0023912

0.3082936000000007

In [59]:
(2*5.7411+2272*0.0023912)*0.002

0.033830012799999996

In [357]:
import  threading 
class MyThread(threading.Thread):
    def __init__(self, func, args=()):
        super(MyThread, self).__init__()
        self.func = func
        self.args = args

    def run(self):
        self.result = self.func(*self.args)

    def get_result(self):
        try:
            return self.result  # 如果子线程不使用join方法，此处可能会报没有self.result的错误
        except Exception:
            return None

In [413]:
def multiple_symbol_price(platform,symbol):
    t1 = MyThread(get_klinedata,args=(platform,'btc_usdt',60))
    t2 = MyThread(get_klinedata, args=(platform,'{}_btc'.format(symbol),60))
    t3 = MyThread(get_klinedata, args=(platform,'{}_usdt'.format(symbol),60))
    t1.start()
    t2.start()
    t3.start()
    t1.join(timeout=20)
    t2.join(timeout=20)
    t3.join(timeout=20)
    p1 = t1.get_result()[1]
    p2 = t2.get_result()[1]
    p3 = t3.get_result()[1]
    return p1, p2, p3
    

In [414]:
p1,p2,p3=multiple_symbol_price('huobi',"eth")

In [417]:
def cal_profit(platform,symbol):
    total_profit=0
    p1,p2,p3=multiple_symbol_price(platform,symbol)
    if len(p2)==len(p3)==len(p1)==1000:
        print("数据获取完整",symbol)
        for i in range(1000):
            forward_profit=p3[i]*(0.998**3)-p1[i]*p2[i]
            backward_profit=p1[i]*p2[i]*(0.998**3)-p3[i]
            if forward_profit>0:
                total_profit+=forward_profit
            elif backward_profit>0:
                total_profit+=backward_profit
    return total_profit

In [418]:
cal_profit('huobi','eth')

数据获取完整 eth


0

In [409]:
symbols=['ht','link','ada','eos','trx','xmr','iota','wbtc','fil','matic','chz','woo','beth','uni','mdx','crv','sun','cvnt','sol',
         'sushi','flow','mln','jst','ar','algo','yfi','ring','aave','pols','zks','snx','pond','firo','nest','ant','pundix','inj',
         'enj','near','hbar','iotx','oxt','ach','grt','axs','avax','kava','glm','fis','bal','badger','mkr','band','uma','comp',
         'lrc','trb','wnxm','yfii','mass','bnt','doge','dbc','qun','lamb','box','nexo','ugas','gas','waxp','btt','aidoc','xvg',
         'mds','xlm','wicc','theta','dat','topc','edu','bft','iris','dta','propy','vet','btm','etn','meet','loom','gxc','atom',
         'zil','eko','knc','qtum','xtz','utk','kmd','iost','cmt','pai','bix','blz','srn','dock','waves','poly','omg','zen','ctxc',
         'mana','qash','chat','itc','ont','ocn','swftc','ela','yee','nano',]

In [410]:
profit_list={}
for i in symbols:
    total_profit=cal_profit('huobi',i)
    if total_profit:
        profit_list[i]=total_profit

数据获取完整 ht
数据获取完整 link
数据获取完整 ada
数据获取完整 eos
数据获取完整 trx
数据获取完整 xmr
数据获取完整 iota
数据获取完整 wbtc
数据获取完整 fil
数据获取完整 matic
数据获取完整 chz
数据获取完整 woo
数据获取完整 beth
数据获取完整 uni
数据获取完整 mdx
数据获取完整 crv
数据获取完整 sun
数据获取完整 sol
数据获取完整 sushi
数据获取完整 flow
数据获取完整 mln
数据获取完整 jst
数据获取完整 ar
数据获取完整 algo
数据获取完整 yfi
数据获取完整 ring
数据获取完整 aave
数据获取完整 pols
数据获取完整 zks
数据获取完整 snx
数据获取完整 pond
数据获取完整 firo
数据获取完整 nest
数据获取完整 ant
数据获取完整 inj
数据获取完整 enj
数据获取完整 near
数据获取完整 hbar
数据获取完整 iotx
数据获取完整 oxt
数据获取完整 ach
数据获取完整 grt
数据获取完整 axs
数据获取完整 avax
数据获取完整 kava
数据获取完整 glm
数据获取完整 fis
数据获取完整 bal
数据获取完整 badger
数据获取完整 mkr
数据获取完整 band
数据获取完整 uma
数据获取完整 comp
数据获取完整 lrc
数据获取完整 trb
数据获取完整 wnxm
数据获取完整 yfii
数据获取完整 mass
数据获取完整 bnt
数据获取完整 doge
数据获取完整 lamb
数据获取完整 nexo
数据获取完整 waxp
数据获取完整 btt
数据获取完整 mds
数据获取完整 xlm
数据获取完整 wicc
数据获取完整 theta
数据获取完整 iris
数据获取完整 dta
数据获取完整 vet
数据获取完整 btm
数据获取完整 loom
数据获取完整 gxc
数据获取完整 atom
数据获取完整 zil
数据获取完整 knc
数据获取完整 qtum
数据获取完整 xtz
数据获取完整 utk
数据获取完整 iost
数据获取完整 cmt
数据获取完整 pai
数据获取完整 bix
数据获取完整 blz
数据获取完整 dock
数据获取完整 waves
数

In [411]:
profit_list

{'xmr': 0.02945225174926236,
 'iota': 0.008616300442268798,
 'wbtc': 297.8149628014362,
 'matic': 0.005658269130844662,
 'chz': 0.0027656716802081927,
 'woo': 0.2846838669239461,
 'beth': 3622.5650415546306,
 'mdx': 0.01780494482584749,
 'crv': 0.006194971304741781,
 'sun': 0.001130632738708174,
 'flow': 0.009727041876695708,
 'mln': 41.872713830579,
 'ar': 19.437355973123267,
 'algo': 0.0015031703004093622,
 'ring': 1.3110423768398969,
 'pols': 0.05513800868093588,
 'zks': 0.22319683017911252,
 'snx': 0.18729764808529303,
 'pond': 0.20551199730204095,
 'firo': 1.2162653659498766,
 'nest': 0.01250661335238678,
 'ant': 0.09469901179538454,
 'inj': 0.014720619081797892,
 'enj': 0.022962237318232548,
 'near': 0.013167030331183494,
 'hbar': 0.005793606935809353,
 'iotx': 0.008576488258330874,
 'oxt': 0.03439949100508072,
 'ach': 0.004124218400555622,
 'axs': 0.15319577659248917,
 'avax': 0.02414161149919991,
 'kava': 0.42469571119587135,
 'glm': 0.11317325907549075,
 'fis': 0.8182538240499

In [412]:
sorted(profit_list.items(), key=lambda item:item[1],reverse=True)

[('yfii', 4449.085735944956),
 ('beth', 3622.5650415546306),
 ('wbtc', 297.8149628014362),
 ('wnxm', 54.95188709284077),
 ('mln', 41.872713830579),
 ('ar', 19.437355973123267),
 ('trb', 6.255446093719655),
 ('uma', 2.5269940497965813),
 ('badger', 2.436756886369656),
 ('ela', 2.0061696791741577),
 ('ring', 1.3110423768398969),
 ('firo', 1.2162653659498766),
 ('mkr', 1.0881421277604204),
 ('fis', 0.8182538240499373),
 ('bix', 0.7843485940814198),
 ('iris', 0.7408364346644689),
 ('waxp', 0.5068504169658908),
 ('kava', 0.42469571119587135),
 ('bal', 0.3863311530022209),
 ('blz', 0.3444775820461592),
 ('mass', 0.32223650827598693),
 ('woo', 0.2846838669239461),
 ('wicc', 0.25196163790487736),
 ('zks', 0.22319683017911252),
 ('knc', 0.21479796845839116),
 ('pond', 0.20551199730204095),
 ('snx', 0.18729764808529303),
 ('axs', 0.15319577659248917),
 ('gxc', 0.12624171953683438),
 ('dock', 0.11772908822058785),
 ('glm', 0.11317325907549075),
 ('ant', 0.09469901179538454),
 ('xlm', 0.0833466649

In [399]:
profit_list['eos']

KeyError: 'eos'